[View in Colaboratory](https://colab.research.google.com/github/itsayushthada/Distributed-Tensorflow/blob/master/Dist-Tensorflow.ipynb)

In [1]:
import tensorflow as tf
from tensorflow.nn import conv2d, relu, max_pool, dropout
from tensorflow.layers import dense, batch_normalization, flatten
from tensorflow.initializers import he_normal
from keras.datasets import cifar10
import numpy as np

Using TensorFlow backend.


In [0]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [3]:
print("Training Set Size: {}".format(x_train.shape))
print("Test Set Size: {}".format(x_test.shape))

Training Set Size: (50000, 32, 32, 3)
Test Set Size: (10000, 32, 32, 3)


In [0]:
def one_hot(x, num_of_class):
    b = np.zeros((x.shape[0], num_of_class))
    b[np.arange(x.shape[0]), x.reshape((-1, ))] = 1
    return b

In [0]:
def normalize(x, is_img=True):
    if is_img:
        return x/255
    else:
        return x / np.max(x[x.shape[0]])

In [0]:
def sample_batch(x_train, y_train, batch_size):
    indices = np.random.randint(0, y_train.shape[0], [batch_size,])
    x_data = np.array(x_train[np.random.randint(0, y_train.shape[0], [batch_size,]), :,:,:])
    y_data = np.array(y_train[np.random.randint(0, y_train.shape[0], [batch_size,]), :])
    return normalize(x_data), one_hot(y_data, 10)

In [0]:
#HYPER_PARAMETERS
batch_size=32
epochs = 10
keep_probability = 0.7
learning_rate = 0.001

In [0]:
tf.reset_default_graph()

#Inputs
x = tf.placeholder(dtype=tf.float16, shape=(None, 32, 32, 3), name="InputX")
y = tf.placeholder(dtype=tf.float16, shape=(None, 10), name="InputY")
#keep_prob = tf.placeholder(dtype=tf.float16, name="DropOutRate")

In [0]:
#Randomly Initilized Variables

conv1_filter = tf.Variable(tf.truncated_normal(shape=[3,3,3,64], dtype=tf.float16, seed=12))
conv2_filter = tf.Variable(tf.truncated_normal(shape=[3,3,64,128], dtype=tf.float16, seed=12))
conv3_filter = tf.Variable(tf.truncated_normal(shape=[3,3,128,256], dtype=tf.float16, seed=12))
conv4_filter = tf.Variable(tf.truncated_normal(shape=[3,3,256,512], dtype=tf.float16, seed=12))
#conv5_filter = tf.Variable(tf.truncated_normal(shape=[1,1,512,1], dtype=tf.float16, seed=12))

In [0]:
#Model Architecture

def model(x):

    with tf.name_scope(name="Layer1"):
        conv1 = conv2d(x, filter=conv1_filter, strides=[1,1,1,1], padding="SAME")
        actv1 = relu(conv1)
        pool1 = max_pool(actv1, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
        bano1 = batch_normalization(pool1)

    with tf.name_scope(name="Layer2"):
        conv2 = conv2d(bano1, filter=conv2_filter, strides=[1,1,1,1], padding="SAME")
        actv2 = relu(conv2)
        pool2 = max_pool(actv2, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
        bano2 = batch_normalization(pool2)

    with tf.name_scope(name="Layer3"):
        conv3 = conv2d(bano2, filter=conv3_filter, strides=[1,1,1,1], padding="SAME")
        actv3 = relu(conv3)
        pool3 = max_pool(actv3, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
        bano3 = batch_normalization(pool3)

    with tf.name_scope(name="Layer4"):
        conv4 = conv2d(bano3, filter=conv4_filter, strides=[1,1,1,1], padding="SAME")
        actv4 = relu(conv4)
        pool4 = max_pool(actv4, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
        bano4 = batch_normalization(pool4)

    """
    with tf.name_scope(name="Layer5"):
        conv5 = conv2d(bano4, filter=conv5_filter, strides=[1,1,1,1], padding="SAME")
        actv5 = relu(conv5)
        bano5 = batch_normalization(actv5)
    """
    
    with tf.name_scope(name="Layer6"): 
        reshape1 = flatten(bano4)
        
    with tf.name_scope(name="Layer7"): 
        dense1 = dense(reshape1, 512)
        #drop1 = dropout(dense1, keep_prob)
        actv6 = relu(dense1)

    with tf.name_scope(name="Layer8"): 
        dense2 = dense(actv6, 128)
        #drop2 = dropout(dense2, keep_prob)
        actv7 = relu(dense2)

    with tf.name_scope(name="Layer9"): 
        dense3 = dense(actv7, 10)
        
    return dense3

In [0]:
#Unnormalized Log Probabilities

pred = model(x)

In [0]:
#Loss & Optimizer

cost=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y, logits=pred))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

In [0]:
#Accuracy

correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, dtype=tf.float16), name="Accuracy")

In [0]:
#Training

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for epoch in range(epochs):
        n_batches = 5
        for batch in range(n_batches):
            x_data, y_data = sample_batch(x_train, y_train, batch_size)   
            loss = sess.run(optimizer, feed_dict = 
                     {
                        x: x_data,
                        y: y_data,
                        #keep_prob: keep_probability
                     }
                    )
            
            acc = sess.run(accuracy, feed_dict = 
                     {
                        x: x_data,
                        y: y_data,
                        #keep_prob: keep_probability
                     }
                    )
        print('Loss: {:>10.4f} Validation Accuracy: {:.6f}'.format(loss, acc))